# San Diego Zipcode Clustering by Decmographic Features

### Setup

In [4]:
import pandas as pd
import numpy as np

### Load & Clean Zipcode Data

In [5]:
# Load csv file of San Diego Zipcode
original_df = pd.read_csv('San Diego 2021 Zipcode.csv')
df = pd.read_csv('San Diego 2021 Zipcode.csv')
print(f'Number of Rows before Drop NaN: {len(df)}')

# drop any row with nan
original_df = original_df.dropna()
df = df.dropna()
print(f'Number of Rows before Drop NaN: {len(df)}')

# Set zipcode column to be the index
original_df.set_index('Zipcode', inplace=True)
df.set_index('Zipcode', inplace=True)

# Show the first few row of the dataframe
df.head()

Number of Rows before Drop NaN: 113
Number of Rows before Drop NaN: 96


,Population,Land Area (Sq. Miles),Population Density (People per Square Mile),Median Age,Population By Age % (Under 18 Years),Population By Age % (18 to 34),Population By Age % (35 to 64),Population By Age % (65 and Over),Male %,White %,...,Carpooled %,Public Transit %,Motorcycle %,Bicycle %,Walked %,Other %.1,Worked at Home %,Health Insurance Coverage %,Married %,Median House Value $
Zipcode,,,,,,,,,,,,,,,,,,,,,
91902,17759.0,11.22,1583.23,45.8,17.62,20.69,38.98,22.72,48.33,37.37,...,5.10,0.97,0.16,0.37,0.00,1.53,10.71,94.78,52.66,732000.0
91910,76291.0,12.59,6061.25,38.2,21.70,23.58,39.95,14.77,49.92,20.32,...,9.54,3.86,0.52,0.29,0.95,0.57,8.75,91.27,48.09,549100.0
91911,88589.0,12.73,6960.37,35.8,24.01,24.85,38.28,12.86,48.26,12.75,...,11.53,4.16,0.18,0.19,1.23,1.04,5.67,90.78,49.91,485300.0
91913,53725.0,9.10,5906.59,35.0,25.91,24.07,39.45,10.58,50.70,13.64,...,9.89,0.87,0.26,0.00,1.29,2.28,11.35,93.92,53.91,578600.0
91914,17742.0,6.64,2670.34,37.1,25.17,20.57,44.35,9.92,52.68,18.14,...,6.05,1.19,0.08,0.43,7.09,3.51,10.57,93.01,61.59,739300.0


### Normalize Data

In [6]:
from sklearn.preprocessing import StandardScaler

# Select the columns you want to normalize
columns_to_normalize = df.columns

# Instantiate the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the selected columns
scaler.fit(df[columns_to_normalize])

# Transform the selected columns with Z-score scaling
df[columns_to_normalize] = scaler.transform(df[columns_to_normalize])

# Display the normalized DataFrame
df

,Population,Land Area (Sq. Miles),Population Density (People per Square Mile),Median Age,Population By Age % (Under 18 Years),Population By Age % (18 to 34),Population By Age % (35 to 64),Population By Age % (65 and Over),Male %,White %,...,Carpooled %,Public Transit %,Motorcycle %,Bicycle %,Walked %,Other %.1,Worked at Home %,Health Insurance Coverage %,Married %,Median House Value $
Zipcode,,,,,,,,,,,,,,,,,,,,,
91902,-0.810811,-0.291858,-0.677925,0.957445,-0.648347,-0.378475,0.052311,-0.052901,-0.644861,-0.600839,...,-0.707695,-0.521183,-0.407319,-0.145270,-0.874593,0.524611,-0.480817,0.467977,0.117090,0.180409
91910,1.847808,-0.279514,0.657912,-0.178781,0.069802,-0.014698,0.296724,-0.113913,-0.124802,-1.425166,...,0.681427,0.860405,0.085917,-0.255952,-0.493590,-0.525066,-0.811411,-0.332152,-0.400228,-0.432641
91911,2.406404,-0.278253,0.926128,-0.537589,0.476401,0.145162,-0.124070,-0.128571,-0.667757,-1.791157,...,1.304029,1.003822,-0.379917,-0.394305,-0.381295,-0.011162,-1.330917,-0.443851,-0.194206,-0.646487
91913,0.822824,-0.310958,0.611775,-0.657192,0.810833,0.046980,0.170738,-0.146069,0.130321,-1.748127,...,0.790930,-0.568989,-0.270309,-0.657175,-0.357231,1.344671,-0.372868,0.271934,0.258588,-0.333762
91914,-0.811584,-0.333122,-0.353629,-0.343235,0.680580,-0.393580,1.405403,-0.151134,0.777942,-1.530563,...,-0.410473,-0.416010,-0.516927,-0.062259,1.968888,2.689569,-0.504431,0.064493,1.127953,0.204877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92081,-0.257349,-0.312400,-0.150555,-0.328285,0.245819,-0.049943,0.541137,-0.126883,0.270966,0.224454,...,-0.645122,-0.640697,0.675060,-0.504987,-0.385305,-0.667210,-0.129982,-0.188539,-0.293821,-0.264044
92083,0.148312,-0.342942,0.940378,-0.941249,0.828434,0.501386,-0.628015,-0.150290,-0.082281,-1.120092,...,-0.216496,-0.043125,0.264030,-0.408140,-0.373274,1.475880,-1.307303,-1.474217,-0.505502,-0.705815
92084,0.642681,-0.137613,-0.626478,-0.642242,0.460559,0.286141,-0.391161,-0.128341,0.513006,-0.391978,...,0.221515,-0.736309,-0.585432,-0.269787,-0.561770,-0.229845,-0.666354,-1.173313,-0.425131,-0.260022


### Clustering by Unsupervised Machine Learning (All features)

In [7]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Standardize features
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

# Choose the number of clusters (n_clusters)
n_clusters = 8
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(scaled_data)

# Get cluster labels and centroids
cluster_labels = kmeans.labels_
centroids = kmeans.cluster_centers_

/Users/lizhihan/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [8]:
# Assign label to each zipcode
original_df['label'] = cluster_labels

##### Checking Cluster Centers

In [9]:
original_df[original_df['label'] == 0].mean()

Population                                      44953.750000
Land Area (Sq. Miles)                              21.086667
Population Density (People per Square Mile)      4525.615417
Median Age                                         34.850000
Population By Age % (Under 18 Years)               23.892083
Population By Age % (18 to 34)                     26.272500
Population By Age % (35 to 64)                     37.304167
Population By Age % (65 and Over)                  66.073333
Male %                                             50.314167
White %                                            38.187500
Black %                                             7.202917
Asian %                                            10.644167
Hispanic %                                         38.505417
Other %                                             5.460000
Less Than High School %                            13.718333
High School Grad or Higher %                       86.281667
Bachelor's Degree or Hig

In [10]:
original_df[original_df['label'] == 1].mean()

Population                                       20655.400
Land Area (Sq. Miles)                               34.202
Population Density (People per Square Mile)       2596.776
Median Age                                          44.690
Population By Age % (Under 18 Years)                20.578
Population By Age % (18 to 34)                      17.199
Population By Age % (35 to 64)                      40.461
Population By Age % (65 and Over)                   21.762
Male %                                              49.594
White %                                             71.134
Black %                                              1.045
Asian %                                              6.688
Hispanic %                                          15.232
Other %                                              5.901
Less Than High School %                              3.905
High School Grad or Higher %                        96.095
Bachelor's Degree or Higher %                       65.6

In [11]:
original_df[original_df['label'] == 2].mean()

Population                                      40472.842105
Land Area (Sq. Miles)                              16.573158
Population Density (People per Square Mile)      3647.235263
Median Age                                         39.273684
Population By Age % (Under 18 Years)               24.120000
Population By Age % (18 to 34)                     20.485263
Population By Age % (35 to 64)                     40.290000
Population By Age % (65 and Over)                  15.103158
Male %                                             49.704737
White %                                            50.474737
Black %                                             3.132105
Asian %                                            19.297895
Hispanic %                                         20.754737
Other %                                             6.340526
Less Than High School %                             5.381053
High School Grad or Higher %                       94.618947
Bachelor's Degree or Hig

In [12]:
original_df[original_df['label'] == 3].mean()

Population                                      59222.200
Land Area (Sq. Miles)                              12.943
Population Density (People per Square Mile)      7524.322
Median Age                                         34.460
Population By Age % (Under 18 Years)               24.469
Population By Age % (18 to 34)                     26.735
Population By Age % (35 to 64)                     36.940
Population By Age % (65 and Over)                  11.854
Male %                                             49.752
White %                                            16.354
Black %                                             5.260
Asian %                                             8.382
Hispanic %                                         66.828
Other %                                             3.176
Less Than High School %                            26.724
High School Grad or Higher %                       73.276
Bachelor's Degree or Higher %                      18.980
Media Househol

In [13]:
original_df[original_df['label'] == 4].mean()

Population                                      33150.000
Land Area (Sq. Miles)                               5.410
Population Density (People per Square Mile)      7456.766
Median Age                                         34.790
Population By Age % (Under 18 Years)               11.852
Population By Age % (18 to 34)                     38.569
Population By Age % (35 to 64)                     37.510
Population By Age % (65 and Over)                  12.069
Male %                                             52.430
White %                                            59.499
Black %                                             4.038
Asian %                                            11.230
Hispanic %                                         19.529
Other %                                             5.704
Less Than High School %                             3.952
High School Grad or Higher %                       96.048
Bachelor's Degree or Higher %                      61.137
Media Househol

In [14]:
original_df[original_df['label'] == 5].mean()

Population                                       2779.00
Land Area (Sq. Miles)                             914.88
Population Density (People per Square Mile)         3.04
Median Age                                         62.80
Population By Age % (Under 18 Years)               10.69
Population By Age % (18 to 34)                      9.86
Population By Age % (35 to 64)                     32.82
Population By Age % (65 and Over)                  46.64
Male %                                             42.75
White %                                            64.66
Black %                                             0.04
Asian %                                             0.04
Hispanic %                                         35.26
Other %                                             0.00
Less Than High School %                             8.56
High School Grad or Higher %                       91.44
Bachelor's Degree or Higher %                      28.53
Media Household Income $       

In [15]:
original_df[original_df['label'] == 6].mean()

Population                                      22821.105263
Land Area (Sq. Miles)                              91.246316
Population Density (People per Square Mile)       860.720000
Median Age                                         45.273684
Population By Age % (Under 18 Years)               19.425789
Population By Age % (18 to 34)                     19.432105
Population By Age % (35 to 64)                     40.251053
Population By Age % (65 and Over)                  20.894211
Male %                                             50.462632
White %                                            65.915789
Black %                                             1.607895
Asian %                                             3.702632
Hispanic %                                         23.519474
Other %                                             5.254211
Less Than High School %                             8.573684
High School Grad or Higher %                       91.426316
Bachelor's Degree or Hig

In [16]:
original_df[original_df['label'] == 7].mean()

Population                                       1336.000000
Land Area (Sq. Miles)                              63.953333
Population Density (People per Square Mile)        35.893333
Median Age                                         45.666667
Population By Age % (Under 18 Years)               21.556667
Population By Age % (18 to 34)                     17.010000
Population By Age % (35 to 64)                     38.213333
Population By Age % (65 and Over)                  23.220000
Male %                                             52.573333
White %                                            39.370000
Black %                                             0.653333
Asian %                                             2.180000
Hispanic %                                         29.063333
Other %                                            28.733333
Less Than High School %                            18.100000
High School Grad or Higher %                       81.900000
Bachelor's Degree or Hig

##### Checking features' influence on clustering

In [17]:
# Assuming 'data' is your preprocessed and scaled data, and 'cluster_labels' are the cluster assignments
variances = np.var(df, axis=0)
feature_ranking = np.argsort(variances)[::-1]  # Features with highest variance first

print("Ranking of features by variance:")
for idx in feature_ranking:
    print(f"{df.columns[idx]}, Variance = {variances[idx]}")

Ranking of features by variance:
Gini Index, Variance = 1.0000000000000007
Population By Age % (18 to 34), Variance = 1.0000000000000007
Motorcycle %, Variance = 1.0000000000000004
Married %, Variance = 1.0000000000000004
Average Commute to Work (min), Variance = 1.0000000000000004
Media Household Income $, Variance = 1.0000000000000004
Bachelor's Degree or Higher %, Variance = 1.0000000000000002
Land Area (Sq. Miles), Variance = 1.0000000000000002
Median Age, Variance = 1.0000000000000002
Population By Age % (35 to 64), Variance = 1.0000000000000002
Asian %, Variance = 1.0000000000000002
Other %, Variance = 1.0000000000000002
High School Grad or Higher %, Variance = 1.0000000000000002
Population, Variance = 1.0000000000000002
Unemployed % , Variance = 1.0000000000000002
Worked at Home %, Variance = 1.0000000000000002
Walked %, Variance = 1.0000000000000002
Public Transit %, Variance = 1.0000000000000002
Black %, Variance = 1.0
Population By Age % (Under 18 Years), Variance = 1.0
Popul

##### Export clustered zipcode data

In [47]:
# Export clustered dataframe as a csv file
csv_filename = 'labeled SD zipcode 4 clusters.csv' 
original_df.to_csv(csv_filename, index=True)

### Unsupervised Clustering (Commute Features)

In [20]:
commute_df = df[['Average Commute to Work (min)', 'Drove Alone %', 'Carpooled %',
       'Public Transit %', 'Motorcycle %', 'Bicycle %', 'Walked %',
       'Other %.1', 'Worked at Home %']]
commute_df

,Average Commute to Work (min),Drove Alone %,Carpooled %,Public Transit %,Motorcycle %,Bicycle %,Walked %,Other %.1,Worked at Home %
Zipcode,,,,,,,,,
91902,-0.126458,1.237120,-0.707695,-0.521183,-0.407319,-0.145270,-0.874593,0.524611,-0.480817
91910,-0.126458,0.395576,0.681427,0.860405,0.085917,-0.255952,-0.493590,-0.525066,-0.811411
91911,0.201650,0.470049,1.304029,1.003822,-0.379917,-0.394305,-0.381295,-0.011162,-1.330917
91913,0.529757,0.179604,0.790930,-0.568989,-0.270309,-0.657175,-0.357231,1.344671,-0.372868
91914,0.037596,-0.265744,-0.410473,-0.416010,-0.516927,-0.062259,1.968888,2.689569,-0.504431
...,...,...,...,...,...,...,...,...,...
92081,-0.290512,0.836456,-0.645122,-0.640697,0.675060,-0.504987,-0.385305,-0.667210,-0.129982
92083,-0.126458,1.223715,-0.216496,-0.043125,0.264030,-0.408140,-0.373274,1.475880,-1.307303
92084,0.037596,1.044980,0.221515,-0.736309,-0.585432,-0.269787,-0.561770,-0.229845,-0.666354


In [21]:
# Standardize features
scaler = StandardScaler()
scaled_data = scaler.fit_transform(commute_df)

# Choose the number of clusters (n_clusters)
n_clusters = 6
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(scaled_data)

# Get cluster labels and centroids
cluster_labels = kmeans.labels_
centroids = kmeans.cluster_centers_

/Users/lizhihan/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [22]:
# Assign label to each zipcode
original_df['label'] = cluster_labels

In [25]:
original_df[original_df['label'] == 0].mean()

Population                                      44097.2400
Land Area (Sq. Miles)                              27.4778
Population Density (People per Square Mile)      4057.3390
Median Age                                         37.6140
Population By Age % (Under 18 Years)               22.1728
Population By Age % (18 to 34)                     24.9458
Population By Age % (35 to 64)                     38.3138
Population By Age % (65 and Over)                  40.2682
Male %                                             50.1294
White %                                            44.0638
Black %                                             5.0150
Asian %                                            10.5918
Hispanic %                                         34.8244
Other %                                             5.5050
Less Than High School %                            11.6800
High School Grad or Higher %                       88.3200
Bachelor's Degree or Higher %                      34.71

In [26]:
original_df[original_df['label'] == 1].mean()

Population                                      30461.571429
Land Area (Sq. Miles)                              11.279286
Population Density (People per Square Mile)      4689.104286
Median Age                                         37.085714
Population By Age % (Under 18 Years)               16.454286
Population By Age % (18 to 34)                     31.404286
Population By Age % (35 to 64)                     36.736429
Population By Age % (65 and Over)                  15.405714
Male %                                             52.257143
White %                                            60.425714
Black %                                             3.105000
Asian %                                            10.395000
Hispanic %                                         20.331429
Other %                                             5.742857
Less Than High School %                             4.317857
High School Grad or Higher %                       95.682143
Bachelor's Degree or Hig

In [27]:
original_df[original_df['label'] == 2].mean()

Population                                      42230.166667
Land Area (Sq. Miles)                              12.788333
Population Density (People per Square Mile)      8549.828333
Median Age                                         33.333333
Population By Age % (Under 18 Years)               27.886667
Population By Age % (18 to 34)                     24.571667
Population By Age % (35 to 64)                     38.313333
Population By Age % (65 and Over)                   9.223333
Male %                                             48.245000
White %                                            17.658333
Black %                                             6.261667
Asian %                                             8.943333
Hispanic %                                         64.448333
Other %                                             2.688333
Less Than High School %                            28.015000
High School Grad or Higher %                       71.985000
Bachelor's Degree or Hig

In [28]:
original_df[original_df['label'] == 3].mean()

Population                                        9060.00
Land Area (Sq. Miles)                               22.39
Population Density (People per Square Mile)        404.59
Median Age                                          46.50
Population By Age % (Under 18 Years)                24.38
Population By Age % (18 to 34)                      11.38
Population By Age % (35 to 64)                      41.08
Population By Age % (65 and Over)                   23.16
Male %                                              45.49
White %                                             72.63
Black %                                              0.00
Asian %                                              6.40
Hispanic %                                          15.97
Other %                                              5.00
Less Than High School %                              1.22
High School Grad or Higher %                        98.78
Bachelor's Degree or Higher %                       71.59
Media Househol

In [29]:
original_df[original_df['label'] == 4].mean()

Population                                      26963.736842
Land Area (Sq. Miles)                              77.171053
Population Density (People per Square Mile)      2622.998947
Median Age                                         43.973684
Population By Age % (Under 18 Years)               21.655789
Population By Age % (18 to 34)                     17.578421
Population By Age % (35 to 64)                     40.788947
Population By Age % (65 and Over)                  19.977895
Male %                                             48.846842
White %                                            59.377368
Black %                                             1.382632
Asian %                                            11.391053
Hispanic %                                         19.812105
Other %                                             8.036842
Less Than High School %                             6.554737
High School Grad or Higher %                       93.445263
Bachelor's Degree or Hig

In [30]:
original_df[original_df['label'] == 5].mean()

Population                                       2077.000000
Land Area (Sq. Miles)                             181.628333
Population Density (People per Square Mile)        16.721667
Median Age                                         50.016667
Population By Age % (Under 18 Years)               17.161667
Population By Age % (18 to 34)                     15.856667
Population By Age % (35 to 64)                     41.033333
Population By Age % (65 and Over)                  25.948333
Male %                                             54.638333
White %                                            70.776667
Black %                                             1.716667
Asian %                                             1.740000
Hispanic %                                         16.455000
Other %                                             9.311667
Less Than High School %                            10.791667
High School Grad or Higher %                       89.208333
Bachelor's Degree or Hig

In [31]:
# Export clustered dataframe as a csv file
csv_filename = 'labeled SD zipcode 6 clusters - commute only.csv' 
original_df.to_csv(csv_filename, index=True)